In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from sklearn.svm import SVC
clf=SVC(probability=True)

In [3]:
from sklearn.datasets import load_iris
dat=load_iris().data
label=load_iris().target

# GridSearch
> [公式ドキュメント](http://scikit-learn.org/stable/modules/grid_search.html#grid-search)

In [4]:
from sklearn import grid_search as grid, cross_validation as cv

#### 分類機のインスタンスはget_params()で設定できるパラメータを表示できる

In [5]:
clf.get_params()

{'C': 1.0,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': None,
 'degree': 3,
 'gamma': 'auto',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': True,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

#### gridで設定したいパラメータを辞書形式で定義する

In [6]:
parameters = {
    'kernel':('linear', 'rbf'),
    'C': np.linspace(1,10,5),
    'gamma' : np.append(
                        np.logspace(-4,1,11).astype('object'),
                        'auto'
                        )
}

#### パラメータグリッドでラップした分類機を作成してフィッティングを行う

> クロスバリデーションまでまとめて実行も可能

> デフォルトはcv=None

In [7]:
cv_method=cv.StratifiedKFold(label,shuffle=True,n_folds=5)
grid_clf=grid.GridSearchCV(clf,parameters,n_jobs=-1,cv=cv_method)

> モデルの詰め合わせになる

In [8]:
grid_clf.param_grid

{'C': array([  1.  ,   3.25,   5.5 ,   7.75,  10.  ]),
 'gamma': array([0.0001, 0.00031622776601683794, 0.001, 0.0031622776601683794, 0.01,
        0.03162277660168379, 0.1, 0.31622776601683794, 1.0,
        3.1622776601683795, 10.0, 'auto'], dtype=object),
 'kernel': ('linear', 'rbf')}

In [9]:
grid_clf.fit(dat,label)

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 2 2], n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'gamma': array([0.0001, 0.00031622776601683794, 0.001, 0.0031622776601683794, 0.01,
       0.03162277660168379, 0.1, 0.31622776601683794, 1.0,
       3.1622776601683795, 10.0, 'auto'], dtype=object), 'C': array([  1.  ,   3.25,   5.5 ,   7.75,  10.  ]), 'kernel': ('linear', 'rbf')},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [10]:
pd.DataFrame(grid_clf.grid_scores_).head()

,parameters,mean_validation_score,cv_validation_scores
0,"{'gamma': 0.0001, 'C': 1.0, 'kernel': 'linear'}",0.973333,"[0.966666666667, 0.966666666667, 1.0, 0.966666..."
1,"{'gamma': 0.0001, 'C': 1.0, 'kernel': 'rbf'}",0.926667,"[0.866666666667, 0.966666666667, 0.9, 0.9, 1.0]"
2,"{'gamma': 0.00031622776601683794, 'C': 1.0, 'k...",0.973333,"[0.966666666667, 0.966666666667, 1.0, 0.966666..."
3,"{'gamma': 0.00031622776601683794, 'C': 1.0, 'k...",0.926667,"[0.866666666667, 0.966666666667, 0.9, 0.9, 1.0]"
4,"{'gamma': 0.001, 'C': 1.0, 'kernel': 'linear'}",0.973333,"[0.966666666667, 0.966666666667, 1.0, 0.966666..."


In [11]:
print(grid_clf.best_params_)
print(grid_clf.best_score_)

{'gamma': 0.1, 'C': 3.25, 'kernel': 'rbf'}
0.986666666667


# ランダムフォレストの場合
> iris(三次元)でランダムフォレスト使わないですが...

In [12]:
from sklearn.ensemble import RandomForestClassifier as RFC
clf_rf=RFC()

In [13]:
print(clf_rf.get_params())

{'max_features': 'auto', 'criterion': 'gini', 'max_leaf_nodes': None, 'n_estimators': 10, 'random_state': None, 'bootstrap': True, 'min_samples_split': 2, 'n_jobs': 1, 'warm_start': False, 'oob_score': False, 'max_depth': None, 'class_weight': None, 'min_weight_fraction_leaf': 0.0, 'min_samples_leaf': 1, 'verbose': 0}


In [14]:
parameters_rf={'n_estimators':[1*2**i for i in range(9)]}
grid_clf_rf=grid.GridSearchCV(clf_rf,parameters_rf,n_jobs=-1,cv=cv_method)
grid_clf_rf.fit(dat,label)

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 2 2], n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [1, 2, 4, 8, 16, 32, 64, 128, 256]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [15]:
grid_clf_rf.best_params_

{'n_estimators': 8}

In [16]:
pd.DataFrame(grid_clf_rf.grid_scores_)

,parameters,mean_validation_score,cv_validation_scores
0,{'n_estimators': 1},0.946667,"[0.966666666667, 0.933333333333, 0.96666666666..."
1,{'n_estimators': 2},0.946667,"[0.933333333333, 0.933333333333, 0.96666666666..."
2,{'n_estimators': 4},0.940000,"[0.933333333333, 0.933333333333, 0.9, 0.966666..."
3,{'n_estimators': 8},0.960000,"[0.966666666667, 0.933333333333, 1.0, 0.933333..."
4,{'n_estimators': 16},0.960000,"[0.966666666667, 0.966666666667, 0.96666666666..."
5,{'n_estimators': 32},0.953333,"[0.933333333333, 0.966666666667, 0.96666666666..."
6,{'n_estimators': 64},0.953333,"[0.933333333333, 0.966666666667, 0.96666666666..."
7,{'n_estimators': 128},0.953333,"[0.933333333333, 0.966666666667, 0.96666666666..."
8,{'n_estimators': 256},0.946667,"[0.9, 0.966666666667, 0.966666666667, 0.933333..."
